# **CSV Analysis**
#https://www.tetranyde.com/blog/langchain-vectordb

In [ ]:
# Install libraries
!pip install langchain chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
def get_insights(question):
    # Load and process the CSV data
    loader = CSVLoader("data.csv")
    documents = loader.load()

    # Create embeddings
    embeddings = OllamaEmbeddings(model="llama2")

    chroma_db = Chroma.from_documents(
        documents, embeddings, persist_directory="./chroma_db"
    )
    chroma_db.persist()

    llm = Ollama(model="llama2")

    prompt_template = PromptTemplate(
        input_variables=["context"],
        template="Given this context: {context}, please directly answer the question: {question}.",
    )

    # Set up the question-answering chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=chroma_db.as_retriever(),
        chain_type_kwargs={"prompt": prompt_template},
    )
    print(chroma_db.as_retriever())
    result = qa_chain({"query": question})
    return result


   ID       heure_debut         heure_fin messagerie utilisation  \
0  14  2/17/25 14:12:50  2/17/25 14:17:49  anonymous         Oui   
1  15  2/17/25 14:11:39  2/17/25 14:24:06  anonymous         Oui   
2  16  2/17/25 14:15:44  2/17/25 14:27:10  anonymous         Oui   
3  17  2/17/25 14:25:24  2/17/25 14:40:47  anonymous         Oui   
4  18  2/17/25 14:42:08  2/17/25 14:42:25  anonymous         Non   

           entite  utilisation_frequence  \
0        Commerce                    3.0   
1        Commerce                    3.0   
2        Commerce                    5.0   
3  Bid management                    5.0   
4             NaN                    NaN   

                                              tache1  \
0  En tant qu'outil d'aide a la redaction de prop...   
1  En tant qu'outil d'aide a la redaction de prop...   
2  En tant qu'outil d'aide a la redaction d'offre...   
3  En tant qu'outil d'aide a la redaction d'offre...   
4                                             

In [ ]:
[Document(page_content='ï»¿Student_Name: Jack\nSubject: Math\nMarks: 90', metadata={'source': 'Marks.csv', 'row': 0}), Document(page_content='ï»¿Student_Name: John\nSubject: Math\nMarks: 60', metadata={'source': 'Marks.csv', 'row': 1}), Document(page_content='ï»¿Student_Name: Mary\nSubject: Math\nMarks: 70', metadata={'source': 'Marks.csv', 'row': 2}), Document(page_content='ï»¿Student_Name: Peter\nSubject: Math\nMarks: 80', metadata={'source': 'Marks.csv', 'row': 3})]

In [ ]:
# Check embeddings
embeddings = OllamaEmbeddings(model="llama2")
query_result = embeddings.embed_query(documents)
print(query_result[:5])

Creating a front


In [ ]:
pip install streamlit

In [ ]:
import streamlit as st
import langchain_helper as lch

st.title("Query Student Records")

question = st.sidebar.text_area(label=f"Ask a question")

if st.sidebar.button("Get Answer"):
    response = lch.get_insights(question)

    st.write("Answer:", response)

In [ ]:
streamlit run streamlit_ui.py